# Data upload

In [1]:

#!pip install googledrivedownloader
!mkdir data_files

import os.path
os.chdir('/content/data_files')

from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1Vy6G1IFiWdTGoHo7YaNVYw7v9Xw3XnuT',
                                    dest_path='/content/data_files/Features_Variant_1.zip',
                                    unzip=True)


Unzipping...Done.


# DataFrame Preparation

In [0]:
import pandas as pd
import numpy as np
import math

data=pd.read_csv('/content/data_files/Features_Variant_1.csv', header=None).drop_duplicates()

In [0]:
#get and fit features names
features_names=np.array(['page_likes_num',#1
               'page_checkins',#2
               'page_talking_about',#3
                'page_cat',#4
                'page_statistics',#5-29 #mean, avg etc.
                'comments_num_before_base_time',#30
                'comments_num_in_last_24_hours',#31 #last day
                'comments_num_in_last_48_to_24_hours',#32 #day before last
                'comments_num_in_first_24_hours',#33
                'comments_difference_in_last_two_days', #34 (32-31)
                'base_time', #35
                'character_num_in_post', #36
                'share_num',#37
                'post_promotion', #38 binary
                'h_local', #39 This describes the H hrs, for which we have the target variable/ comments received. 
                'post_published_weekday', #40-46 This represents the day(Sunday...Saturday) on which the post was published. 
                'base_ditetime_weekday', #47-53 This represents the day(Sunday...Saturday) on selected base Date/Time. 
                'target' #54 The no of comments in next H hrs(H is given in Feature no 39).                
               ])

for index in range(5,29):
    features_names=np.insert(features_names, index, features_names[4]+'_'+str(index-4))
    
weekday=('sunday', 'monday','tuesday', 'wednesday', 'thursday', 'friday', 'saturday')    

for index in range(40,47):
    features_names=np.insert(features_names,index, features_names[39]+'_'+ weekday[index-40])
features_names=np.delete(features_names, 39)

for index in range(47,54):
    features_names=np.insert(features_names,index, features_names[46]+'_'+ weekday[index-47])
features_names=np.delete(features_names, 46)

data.columns=features_names

In [4]:
y=data.nunique(axis=0).sort_values()
print(y.head())
data=data.drop(columns=['post_promotion']) #post promotion has only zero values we can drop this col because it isn't informative

post_promotion                      1
post_published_weekday_sunday       2
post_published_weekday_monday       2
post_published_weekday_tuesday      2
post_published_weekday_wednesday    2
dtype: int64


# Data preparation

In [0]:
#'page_cat' to dummy because it is a category
data=pd.concat([data,pd.get_dummies(data['page_cat'], prefix='page_cat')], axis=1)
data=data.drop(columns=['page_cat'])

In [0]:
#data shuffling
from sklearn.utils import shuffle
data=shuffle(data).reset_index(drop=True)

In [0]:
def normalize(df):
    std=np.std(df, ddof=1).replace(0, 1)
    mean=np.mean(df)
    return (df-mean)/std, mean, std

def normalize_with_params(df, mean, std):
    return (df-mean)/std

# learning

In [0]:
def custom_RMSE(target, pred, gamma, w):
    return np.sqrt(
        np.sum((target-pred)**2)/len(pred)+gamma*np.linalg.norm(w)
    )

def get_para_w(old_w, df, pred, lam, gamma, indices):
    #indices=np.random.choice(df.shape[0]-1,terms_num,replace=False)
    old_w=old_w+2*lam*(
          (np.dot((df.loc[indices,'target']-pred[indices]).values,df.iloc[indices,:-1].values))/len(pred)
        )
    return old_w

def get_para_w0(old_w0, df, pred, lam, gamma, indices):
    return old_w0+2*lam*(
        (np.sum(df.loc[indices,'target']-pred[indices]))/len(pred))

def get_prediction(w,w0,x_df):
    return np.dot(x_df, w)+w0
    
def gradient_descend(df, lam, gamma, terms_num, max_iter):
    df=df.reset_index(drop=True)
    w=np.array([0]*(df.shape[1]-1), dtype=float) #one column is the target
    w0=0.0
    prediction=get_prediction(w,w0,df.iloc[:,:-1])
    minRMSE=custom_RMSE(df['target'], prediction, gamma, w)

    best_params=np.append(w,w0)
    best_pred=prediction

    curr_err=10
    err=1e-4
    
    iter_num=0
    while (curr_err>err) & (iter_num<max_iter) :    
        iter_num+=1
        df=shuffle(df)
        prediction=get_prediction(w,w0,df.iloc[:,:-1])
        curr_err=prediction.copy()
        
        for batch_counter in range(0, math.ceil(df.shape[0]/terms_num)):
            indices=np.array(df.iloc[batch_counter*terms_num:(batch_counter+1)*terms_num].index)            
            w=get_para_w(w, df, prediction, lam, gamma, indices)
            w0=get_para_w0(w0, df, prediction, lam, gamma, indices)            
            prediction=get_prediction(w,w0,df.iloc[:,:-1])

            if custom_RMSE(df['target'], prediction, gamma, w)<minRMSE:
                minRMSE=custom_RMSE(df['target'], prediction, gamma, w)
                best_params=np.append(w,w0)
                best_pred=prediction
        curr_err=np.linalg.norm(curr_err-prediction)
            
    print('Iterations number is:' +str(iter_num))
    if iter_num==max_iter:
        print('The maximum number of iterations was reached.')
    print('RMSE is:'+str(minRMSE))
    print('R2 is:'+str(custom_R2(df.iloc[:,-1], best_pred)))
    
    return best_params

In [0]:
#set algo params:
lam=5e-4
gamma=0 #now we needn't it
terms_num=100
max_iter=50

# cross-val

In [0]:
def custom_R2(target, pred):
    return 1-(np.sum((target-pred)**2))/(np.sum((target-np.mean(target))**2))

In [11]:
res=pd.DataFrame()

parts=range(1,6)
idxs=[round(data.shape[0]/5)*(i-1) for i in range(1,6)]
idxs.append(data.shape[0])

for part in parts:
    #separate dataset
    test_part=data[idxs[part-1]:idxs[part]]
    train_part=data.drop(data.index[idxs[part-1]:idxs[part]])
    #features normalization (xi-mean)/std_err:
    train_part.iloc[:,:-1], mean, std=normalize(train_part.iloc[:,:-1])#fit and transform train
    test_part.iloc[:,:-1]=normalize_with_params(test_part.iloc[:,:-1], mean, std) #transform test
    params=gradient_descend(train_part, lam, gamma, terms_num, max_iter) #model    
    test_pred=get_prediction(params[:-1],params[-1],test_part.iloc[:,:-1])
    RMSE=custom_RMSE(test_part['target'], test_pred, gamma, params[:-1])
    r2=custom_R2(test_part['target'], test_pred)
    params=np.insert(params, 0 , [RMSE,r2])
    res[part]=params

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Iterations number is:50
The maximum number of iterations was reached.
RMSE is:0.9997339912519347
R2 is:-0.0016002975011735199
Iterations number is:50
The maximum number of iterations was reached.
RMSE is:0.9999179514598752
R2 is:-0.0014085880997212197
Iterations number is:50
The maximum number of iterations was reached.
RMSE is:0.9998938493347957
R2 is:-0.0017133504058663984
Iterations number is:50
The maximum number of iterations was reached.
RMSE is:0.9998363876055629
R2 is:-0.0018796021080604852
Iterations number is:50
The maximum number of iterations was reached.
RMSE is:0.9998726994413694
R2 is:-0.0015583151967220044


In [0]:
text_idxs=np.array(['w'+str(i) for i in range(1,res.shape[0]-2)])
text_idxs=np.append(text_idxs,'w0')
text_idxs=np.insert(text_idxs, 0, ['RMSE', 'r2'])
res.index=text_idxs

res['mean']=np.mean(res, axis=1)
res['std']=np.std(res.iloc[:,:-1], ddof=1, axis=1)

In [13]:
res.iloc[:2]

,1,2,3,4,5,mean,std
RMSE,1.227466,1.021992,0.897118,1.039163,0.793094,0.995766,0.163603
r2,0.000167,0.000111,0.000353,0.000737,0.025681,0.005410,0.011335
